```
from pathlib import Path

folder_path = Path("your/folder/path")
csv_files = [f for f in folder_path.glob("*.csv") if f.is_file() and f.stat().st_size > 0]

print(csv_files)
```

In [151]:
import logging

from collections import defaultdict
from csv import DictReader, reader
from datetime import datetime
from pathlib import Path
from prettytable import PrettyTable, TableStyle
from typing import Dict, List

In [ ]:
class Schedule:
	def __init__(self, employees: dict, template: dict, weekday: str):
		self.employees = employees
		self.template = template
		self.weekday = weekday

		self.weekday_time_slots = {
			"9-11": [datetime.strptime("0900", "%H%M").time(), datetime.strptime("1100", "%H%M")],
			"11-1": [datetime.strptime("1100", "%H%M").time(), datetime.strptime("1300", "%H%M")],
			"1-2": [datetime.strptime("1300", "%H%M").time(), datetime.strptime("1400", "%H%M")],
			"2-4": [datetime.strptime("1400", "%H%M").time(), datetime.strptime("1600", "%H%M")],
			"4-6": [datetime.strptime("1600", "%H%M").time(), datetime.strptime("1800", "%H%M")],
			"6-8": [datetime.strptime("1800", "%H%M").time(), datetime.strptime("2000", "%H%M")]
		}

		self.weekend_time_slots = {
			"9-10": [datetime.strptime("0900", "%H%M").time(), datetime.strptime("1000", "%H%M")],
			"10-11": [datetime.strptime("1000", "%H%M").time(), datetime.strptime("1100", "%H%M")],
			"11-1": [datetime.strptime("1100", "%H%M").time(), datetime.strptime("1300", "%H%M")],
			"1-2": [datetime.strptime("1300", "%H%M").time(), datetime.strptime("1400", "%H%M")],
			"2-4": [datetime.strptime("1400", "%H%M").time(), datetime.strptime("1600", "%H%M")],
			"4-6": [datetime.strptime("1600", "%H%M").time(), datetime.strptime("1800", "%H%M")]
		}

		self.sunday_time_slots = {
			"2-3": [datetime.strptime("1400", "%H%M").time(), datetime.strptime("1500", "%H%M")],
			"3-4": [datetime.strptime("1500", "%H%M").time(), datetime.strptime("1600", "%H%M")],
			"4-5": [datetime.strptime("1600", "%H%M").time(), datetime.strptime("1700", "%H%M")],
			"5-6": [datetime.strptime("1700", "%H%M").time(), datetime.strptime("1800", "%H%M")]
		}
		
		self.header_table = PrettyTable(["who works today", "lunch breaks", "schedule changes"])
		self.content_table = None

		if weekday in ["monday", "tuesday", "wednesday", "thursday"]:
			self.content_table = list(self.weekday_time_slots.keys())
		elif weekday in ["friday", "saturday"]:
			self.content_table = list(self.weekend_time_slots.keys())
		elif weekday == "sunday":
			self.content_table = list(self.sunday_time_slots.keys())
	
	def _get_workers_by_position(self, position: str):
		working_employees = []
		hours = set()
		for employee in self.employees.values():
			if employee.position == position and employee.hours[self.weekday + "-hours"]:
				working_employees.append(employee)
				hours.add(h for h in employee.hours[self.weekday + "-hours"]) # fix this
				# print(employee.hours[self.weekday + "-hours"])
		print(list(hours))
		return (working_employees, hours)

In [153]:
class Template:
	def __init__(self, location: Dict[str, List[str]]):
		"""
		Initialize a `Template` object.

		Args:
			location (Dict[str, List[str]]): Each location and associated working employees with optional decoding.
		"""

		self.location = location

```csv
puw,fl,sp1a,sp1b,sp2a,sp2b
eric,yvette,wilbur,sean,chris,jasper
wilbur,none,samuel,eric/1200,sean,corey/1200
antwon,none,corey,eric,rob,none
chris/1530/christa,none,lee,none,antwon,none
sean/1730/saxon,jordan,christa,derrick/1630,wanda,eric/1730
jordan,lee,wanda,saxon,derrick,christa
```

In [154]:
class Employee:
	"""Creates an employee object with additional functions."""

	def __init__(
		self,
		name: str,
		rank: int,
		position: str,
		hours: Dict[str, List[str]],
		lunch_hours: Dict[str, List[str]],
	):
		"""
		Initialize an `Employee` object.

		Args:
			name (str): Full name of the employee.
			rank (int): Their rank against the employee's colleagues.
			position (str): The employee's position, typically *manager*, *assistant manager*, *supervisor*, *full time*, *part time*, *shelver*, *full time security*, and *part time security*.
			hours (Dict[str, List[str]]): Dictionary mapping days to work, e.g. `{"monday": [0900, 1730]}`.
			lunch_hours (Dict[str, List[str]]): Dictionary mapping days to work, e.g. `{"monday": [1200, 1300]}`.
		"""

		self.name = name
		self.first_name = self._extract_first_name(name)
		self.initials = self._generate_initials(name)

		self.rank = rank
		self.position = position

		self.hours = self._convert_to_datetime(hours)
		self.phours = self._convert_to_ptime(hours)
		self.lunch_hours = self._convert_to_datetime(lunch_hours)
		self.plunch_hours = self._convert_to_ptime(lunch_hours)

		self.programs = None

	def _extract_first_name(self, full_name: str) -> str:
		"""Extract the first name from a full name string."""

		return full_name.split()[0]

	def _generate_initials(self, name: str) -> str:
		"""Generate initials from a full name, handling hyphenated names."""

		return "".join(i[0] for i in name.replace("-", " ").split())

	def _convert_to_datetime(self, hours):
		"""Convert string times (`"0800"`) to `datetime.time` objects."""

		result = {}
		for day, times in hours.items():
			result[day] = [datetime.strptime(t.zfill(4), "%H%M").time() for t in times]
		return result

	def _convert_to_ptime(self, hours):
		"""Convert string times to readable 12-hour format (`"0800" -> "8"`)."""

		result = {}
		for day, times in hours.items():
			dt = [datetime.strptime(t.zfill(4), "%H%M") for t in times]
			hours = [str(t.hour % 12) or "12" for t in dt]
			mins = ["" if t.minute == 0 else str(t.minute) for t in dt]
			ft = [":".join([str(h), f"{m}"]).rstrip(":") for h, m in zip(hours, mins)]
			result[day] = ft
		return result

	def _get_schedule_summary(self) -> Dict[str, int]:
		"""
		Get a summary of the employee's coverage.

		Returns:
			A `dict` with counts of work days and lunch days.
		"""

		return {
			"work_days": len(self.hours.keys()),
			"lunch_days": len(self.lunch_hours.keys()),
		}

	def __str__(self) -> str:
		"""
		Create a formatted string table of basic employee data.
		"""

		name_info = f"Abbv: {self.initials}\t || First: {self.first_name}"
		pos_info = f"Rank: {self.rank:>2}\t || Position: {self.position}"

		schedule_summary = self._get_schedule_summary()
		schedule_info = f"Hrs: {schedule_summary['work_days']:>3}\t || Lunch hrs: {schedule_summary['lunch_days']}"

		lines = [name_info, pos_info, schedule_info]
		max_width = max(len(line.expandtabs()) for line in lines)

		name_padding = max(0, (max_width - len(self.name)) // 2)
		centered_name = " " * name_padding + self.name

		divider = "=" * max_width

		formatted_sections = [
			divider,
			centered_name,
			divider,
			name_info,
			pos_info,
			schedule_info,
		]

		return "\n".join(formatted_sections) + "\n"

In [155]:
class Loader:
	"""Loads and converts employees to `Employee` objects and templates as `Template` objects"""
	
	def __init__(self):
		self.employees = {}
		self.templates = {}
		
		self.key = None
		self.locations_key = {
			"puw": "pick-up window",
			"fl": "floor lead",
			"sp1a": "service point 1",
			"sp1b": "service point 1",
			"sp2a": "service point 2",
			"sp2b": "service point 2"
		}
	
	def load_employees(self, employees_path: str, employees_key: str = ""):
		try:
			key_map = {}
			with open(employees_key, "r", encoding="utf-8") as keyfile:
				for line in keyfile:
					encoded, decoded = line.split("= ")
					key_map[encoded.strip()] = decoded.strip()
			self.key = key_map
		except FileNotFoundError as e:
			logging.exception(f"Path: '{employees_key}' does not exist. {e}")
			self.key = None
		
		employees = {}
		try:
			fields = []
			rows = []
			with open(employees_path, "r", encoding="utf-8") as csvfile:
				csvreader = reader(csvfile)
				fields = next(csvreader)
				for row in csvreader:
					rows.append(row)
			
			for row in rows:
				name = self.key.get(row[0], row[0]) if self.key else row[0]
				rank, position = row[1], row[2]
				hours, lunch_hours = {}, {}
				
				for i in range(3, len(fields)):
					key, value = fields[i], row[i]
					if "-hours" in key and value != "off":
						hours[key] = value.split("-")
					if "-lunch" in key and value != "none":
						lunch_hours[key] = value.split("-")
				
				employee = Employee(name, rank, position, hours, lunch_hours)
				employees[employee.first_name] = employee
			
			return employees
			
		except FileNotFoundError as e:
			logging.exception(f"Path: '{employees_path}' does not exist. {e}")
		except Exception as e:
			logging.exception(f"Failed to load '{employees_path}': {e}")
		
		return employees

	def load_templates(self, templates_path: str):
		templates_dir = Path(templates_path)
		csv_files = [str(f).replace("\\", "/") for f in templates_dir.glob("*.csv") if f.is_file() and f.stat().st_size > 0]

		if len(csv_files) > 0:
			# self.templates = [self._load_template(f) for f in csv_files]
			for f in csv_files:
				template_day = Path(f).stem.replace("_template", "")
				self.templates[template_day] = self._load_template(f)
			
			return self.templates


	def _load_template(self, template_path: str):
		data = defaultdict(list)
		try:
			fields = []
			rows = []
			with open(template_path, "r", encoding="utf-8") as csvfile:
				csvreader = DictReader(csvfile)
				for row in csvreader:
					for key, value in row.items():
						key = self.locations_key[key]
						working_employees = []
						if self.key:
							if "/" in value:
								raw_pairs = value.split("/")
								employee_pairs = [self._get_decoded_first_name(e) for e in raw_pairs]
								if employee_pairs[0].isdigit():
									employee_pairs[0] = self._convert_to_datetime(employee_pairs[0])
								elif employee_pairs[1].isdigit():
									employee_pairs[1] = self._convert_to_datetime(employee_pairs[1])
								data[key].append(employee_pairs)
							else:
								data[key].append(self._get_decoded_first_name(value))
			
			result = dict(data)
			return result
		
		except FileNotFoundError as e:
			logging.exception(f"Path: '{template_path}' does not exist. {e}")
		except Exception as e:
			logging.exception(f"Failed to load '{template_path}': {e}")
	
	def _get_decoded_first_name(self, query: str) -> str:
		query = query.lower()
		if self.key:
			for k, v in self.key.items():
				first_name = k.split()[0].lower()
				if first_name == query:
					return v
		return query if query != "none" else ""

	def _convert_to_datetime(self, hour: str):
		"""Convert a string time (`"0800"`) to `datetime.time` a object."""

		return datetime.strptime(hour.zfill(4), "%H%M").time()

In [156]:
loader = Loader()

employees = loader.load_employees("../models/employees.csv", "../models/employees_key.txt")
# print(employees["Chris"])
# print(employees["Jess"])
# print(employees["Sonaite"])

templates = loader.load_templates("../models/templates")
# print(templates["tuesday"])

tuesday = Schedule(employees, templates["tuesday"], "tuesday")
test = tuesday._get_workers_by_position("full time")

print(test)

[<generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f69f400>, <generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f896e00>, <generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f69ec80>, <generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f896ec0>, <generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f8962c0>, <generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f896500>, <generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10f896140>]
([<__main__.Employee object at 0x10f117df0>, <__main__.Employee object at 0x10f9af550>, <__main__.Employee object at 0x10fa93d50>, <__main__.Employee object at 0x10fbe23f0>, <__main__.Employee object at 0x10fbe24e0>, <__main__.Employee object at 0x10f8442f0>, <__main__.Employee object at 0x10f846890>], {<generator object Schedule._get_workers_by_position.<locals>.<genexpr> at 0x10